# Atrial Fibrillation Detection

This project aims to detect periods of AF from preprocessed ECG data by detecting irregular R-R intervals in the ECG signals. Different Machine Learning Classifiers are used to build an ML model using the given data. The performance of these various classifiers are analysed to determine the best classifier for this data. This is an interactive notebook that displays the results of our work. The entire source code can be found at the [GitHub](https://github.com/rushvanth/AFDetection) repository. 

It is highly recommended to run this Notebook in [Colab](https://colab.research.google.com/) in a GPU backed runtime. This drastically reduces run times while training models. In some cases, training time can be reduced from >5 hours (CPU processing - i7 1135G7) to ~4 minutes(GPU processing - NVIDIA T4). 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rushvanth/AFDetection/)


### RAPIDS Setup on Colab

Borrowed from [Rapids.ai](https://colab.research.google.com/drive/1rY7Ln6rEE1pOlfSHCYOVaqt8OvDO35J0#forceEdit=true&offline=true&sandboxMode=true)

#### Environment Sanity Check

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [ ]:
!nvidia-smi

#### Setup:
Set up script installs
1. Updates gcc in Colab
1. Installs Conda
1. Install RAPIDS' current stable version of its libraries, as well as some external libraries including:
     1. cuDF
     2. cuML
     3. cuGraph
     4. cuSpatial
     5. cuSignal
     6. BlazingSQL
     7. xgboost
1. Copy RAPIDS .so files into current working directory, a neccessary workaround for RAPIDS+Colab integration.

In [ ]:
# This will get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

In [ ]:
# This will update the Colab environment and restart the kernel. Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

In [ ]:
# This will install CondaColab. This will restart your kernel one last time. Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

In [ ]:
# You can now run the rest of the cells as normal
import condacolab
condacolab.check()

In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

### Utility Functions

In [ ]:
# Function to generate visualizations of classification results

import os
from sklearn import metrics
import  scikitplot as skplt
import matplotlib.pyplot as plt

def svm_feature_importance(coef, names, fig_path, plot_title):
    """Visualize the feature importance of the SVM classifier."""
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    # Plot the feature importance
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.title(f'{plot_title} Feature Importance')
    plt.savefig(os.path.join(fig_path, 'feature_importance.png'))


def visualize_results(results, classifier_name):
    """Visualize ROC Curve, Confusion Matrix, Classification Report and Feature Importance."""
    feature_names = results['x_train'].columns.tolist()
    # Derive plot titles from classifier name. Make first letter uppercase and replace underscores with spaces
    plot_title = classifier_name.title().replace('_', ' ')
    # Join classifier_name with images_path
    fig_path = os.path.join('images', classifier_name)
    # Create directory if it doesn't exist
    if not os.path.exists(fig_path):
        os.makedirs(fig_path)
    # Feature Importance
    # If Classifier name has svm, use a different method to visualize feature importance
    if 'linear_svm' == classifier_name:
        svm_feature_importance(results['model'].coef_[0], feature_names, fig_path, plot_title)
    elif 'decision_tree' in classifier_name:
        skplt.estimators.plot_feature_importances(results['model'], feature_names=results['x_train'].columns, figsize=(14,6))
        plt.title(f'{plot_title} Feature Importance')
        plt.savefig(os.path.join(fig_path, 'feature_importance.png'))
    # ROC Curve
    roc_curve = metrics.RocCurveDisplay.from_predictions(results['y_test'], results['y_pred'])
    roc_curve.plot()
    plt.title(f'{plot_title} ROC Curve')
    plt.savefig(os.path.join(fig_path, 'roc_curve.png'))
    # Confusion Matrix
    skplt.metrics.plot_confusion_matrix(results['y_test'], results['y_pred'])
    plt.title(f'{plot_title} Confusion Matrix')
    plt.savefig(os.path.join(fig_path, 'confusion_matrix.png'))
    # Plot Precision-Recall Curve
    precision, recall, _ = metrics.precision_recall_curve(results['y_test'], results['y_pred'])
    precision_recall_display = metrics.PrecisionRecallDisplay(precision, recall)
    precision_recall_display.plot()
    plt.title(f'{plot_title} Precision-Recall Curve')
    plt.savefig(os.path.join(fig_path, 'precision_recall_curve.png'))

In [3]:
# Function to generate reports for each model

def generate_report(clf, classifier_name):

    print(f"{classifier_name} Results\n" + "-" * 50 + "\n")
    print('Accuracy: {:.4F} \n'.format(metrics.accuracy_score(clf['y_test'], clf['y_pred']) * 100))
    print('Confusion Matrix: \n', metrics.confusion_matrix(clf['y_test'], clf['y_pred']))
    print('Area under curve: {:.4F} \n'.format(metrics.roc_auc_score(clf['y_test'], clf['y_pred'])))
    print(metrics.classification_report(clf['y_test'], clf['y_pred']))
    print("-" * 50 + "\n")

### Generate Metadata for Dataset

In [ ]:
"""Load Data and generate some metadata regarding the classes"""
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Read the file from the data directory
af_data = pd.read_csv('data/Preprocessed_AFData.csv')
# Print some metadata about the classes
(unique,counts) = np.unique(af_data['Control'],return_counts=True)
print("Metadata:\n" + "-"*50 + "\n")
print(f"Classes: {str(unique)} \n")
print("Class Labels: \n 0 - Non AF\n 1 - AF\n")
print(f"Data in the 'Control' column: {dict(zip(unique,counts))} \n")
print(f"Ratio of occurrences of each class: {dict(zip(unique,counts/len(af_data['Control'])))}\n")
print("-"*50 + "\n")
# Display counts on a graph and save it
target_variables = ['Non-AF','AF']
sns.set_theme(style='whitegrid')
sns.barplot(x=target_variables,y=counts)
plt.title('Count of AF and Non-AF occurrences in the Preprocessed data')
plt.ylabel('Count')
for i,_ in enumerate(counts):
    plt.text(i-0.25, counts[i]+0.5, counts[i], color='black', fontweight='bold')
img_file_path = 'images/general/count_of_AF_and_Non_AF_occurrences.png'
plt.savefig(img_file_path)

In [7]:
# Split the data
from sklearn.model_selection import train_test_split

X = af_data.drop(['Control'],axis=1)
y = af_data['Control']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)